In [234]:
import pandas as pd

In [235]:
dataset=pd.read_csv(r"C:\Users\wwrao\OneDrive\Documents\cleaned_file.csv")

In [236]:
dataset.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508,128,360,1,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0,66,360,1,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358,120,360,1,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0,141,360,1,Urban,Y
4,LP001011,Male,Yes,2,Graduate,Yes,5417,4196,267,360,1,Urban,Y


In [237]:
dataset.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [238]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder
labelencoder=LabelEncoder()
dataset["Loan_Status"]=labelencoder.fit_transform(dataset["Loan_Status"])

In [239]:
dataset=dataset.drop(columns=["Loan_ID"])

In [240]:
x=dataset.iloc[:,:-1]
y=dataset.iloc[:,-1]


In [241]:
from sklearn.model_selection import train_test_split

In [242]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=42)
y_train.head()

9     1
0     0
4     1
16    0
17    0
Name: Loan_Status, dtype: int32

In [243]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((25, 11), (9, 11), (25,), (9,))

In [244]:
from sklearn.compose import ColumnTransformer
x_train.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
9,Male,No,2,Graduate,No,3200,700,70,360,1,Urban
0,Male,Yes,1,Graduate,No,4583,1508,128,360,1,Rural
4,Male,Yes,2,Graduate,Yes,5417,4196,267,360,1,Urban
16,Female,No,0,Graduate,No,3510,0,76,360,0,Urban
17,Male,Yes,0,Not Graduate,No,4887,0,133,360,1,Rural


In [245]:
ct1=ColumnTransformer(transformers=[
    ("t1",OrdinalEncoder(categories=[["Male","Female"],["Yes","No"]]),[0,1])
],remainder="passthrough")
ct1

ColumnTransformer(remainder='passthrough',
                  transformers=[('t1',
                                 OrdinalEncoder(categories=[['Male', 'Female'],
                                                            ['Yes', 'No']]),
                                 [0, 1])])

In [246]:
ct2=ColumnTransformer(transformers=[
    ("t2",OrdinalEncoder(categories=[["Graduate","Not Graduate"],["Yes","No"]]),[3,4])
],remainder="passthrough")
ct2

ColumnTransformer(remainder='passthrough',
                  transformers=[('t2',
                                 OrdinalEncoder(categories=[['Graduate',
                                                             'Not Graduate'],
                                                            ['Yes', 'No']]),
                                 [3, 4])])

In [247]:
ct3=ColumnTransformer(transformers=[
    ("t3",OneHotEncoder(drop="first",sparse_output=False),[10])
],remainder="passthrough")
ct3

ColumnTransformer(remainder='passthrough',
                  transformers=[('t3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 [10])])

In [248]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier


In [249]:
pipe=Pipeline(steps=[
    ("t1",ct1),
    ("t2",ct2),
    ("t3",ct3),
    ("tree",DecisionTreeClassifier(max_depth=2))
   
])

In [250]:
pipe

Pipeline(steps=[('t1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t1',
                                                  OrdinalEncoder(categories=[['Male',
                                                                              'Female'],
                                                                             ['Yes',
                                                                              'No']]),
                                                  [0, 1])])),
                ('t2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t2',
                                                  OrdinalEncoder(categories=[['Graduate',
                                                                              'Not '
                                                                              'Graduate'],
                                                                             ['Yes',
                                                                              'No']]),
                                                  [3, 4])])),
                ('t3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t3',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  [10])])),
                ('tree', DecisionTreeClassifier(max_depth=2))])

In [251]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('t1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t1',
                                                  OrdinalEncoder(categories=[['Male',
                                                                              'Female'],
                                                                             ['Yes',
                                                                              'No']]),
                                                  [0, 1])])),
                ('t2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t2',
                                                  OrdinalEncoder(categories=[['Graduate',
                                                                              'Not '
                                                                              'Graduate'],
                                                                             ['Yes',
                                                                              'No']]),
                                                  [3, 4])])),
                ('t3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('t3',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  [10])])),
                ('tree', DecisionTreeClassifier(max_depth=2))])

In [252]:
y_pred=pipe.predict(x_test)
y_pred

array([1, 0, 1, 1, 1, 1, 0, 0, 1])

In [253]:
y_test

15    1
19    0
27    1
26    1
8     0
24    1
21    0
12    1
32    1
Name: Loan_Status, dtype: int32

In [254]:
from sklearn.metrics import accuracy_score


In [255]:
accuracy_score(y_test,y_pred)

0.7777777777777778

In [260]:
file_name="file1.pkl"
file="file2.pkl"

In [261]:
import pickle

In [262]:
pickle.dump(pipe,open(file,"wb"))
pickle.dump(labelencoder,open(file_name,"wb"))